Κείμενο 1:

“Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes.Thank your message to show our words to the doctor, as his next contract checking, to all of us.I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication”

Κείμενο 2:

“During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets”

Παραδοτέο 1: Ανακατασκευή Κειμένου

Απο τα παραπάνω κείμενα σας ζητείται να υλοποιήσετε τα εξής:

A. Aνακατασκευή 2 προτάσεων της επιλογής σας με αυτόματο που θα διαμορφώσετε εσείς.

In [3]:
import math
import random

# ---------------------------
# Tokenization / utils
# ---------------------------
def tokenize(sent):
    s = sent.replace(',', ' , ')
    toks = [t.strip().lower() for t in s.split() if t.strip() != ',']
    return toks

def detokenize(tokens):
    if not tokens:
        return ""
    toks = [t.upper() if t == 'i' else t for t in tokens]
    s = " ".join(toks)
    if ' with ' in s:
        s = s.replace(' with ', ', with ', 1)
    if not s.endswith('.'):
        s = s + '.'
    s = s.replace('..', '.')
    return s

def levenshtein(a, b):
    la, lb = len(a), len(b)
    dp = [[0]*(lb+1) for _ in range(la+1)]
    for i in range(la+1):
        dp[i][0] = i
    for j in range(lb+1):
        dp[0][j] = j
    for i in range(1, la+1):
        for j in range(1, lb+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(dp[i-1][j] + 1,
                           dp[i][j-1] + 1,
                           dp[i-1][j-1] + cost)
    return dp[la][lb]

VOCAB = [
    'i', 'you', 'we', 'it',
    'hope', 'enjoy', 'too', 'with', 'my', 'deepest', 'wishes',
    'got', 'your', 'message', 'to', 'review', 'the', 'approved', 'content',
    'see', 'this', 'a', 'is', 'was', 'that', 'part', 'missed', 'apologize'
]

BIGRAM = {
    ('<s>', 'i'): 0.6,
    ('<s>', 'hope'): 0.05,
    ('<s>', 'got'): 0.05,
    ('i', 'hope'): 0.6,
    ('hope', 'you'): 0.5,
    ('you', 'enjoy'): 0.55,
    ('enjoy', 'it'): 0.6,
    ('it', 'too'): 0.45,
    ('too', 'with'): 0.35,
    ('with', 'my'): 0.7,
    ('my', 'deepest'): 0.75,
    ('deepest', 'wishes'): 0.75,

    # chains for sentence 2
    ('i', 'got'): 0.55,
    ('got', 'your'): 0.5,
    ('your', 'message'): 0.55,
    ('message', 'to'): 0.3,
    ('to', 'review'): 0.5,
    ('review', 'the'): 0.45,
    ('the', 'approved'): 0.5,
    ('approved', 'content'): 0.7
}

BIGRAM_DEFAULT = 1e-3

PCFG_PROBS = {
    'S->NP_VP': 0.8,
    'S->VP_NP': 0.2,
    'NP->Pron': 0.4,
    'NP->Det_N1': 0.6,
    'N1->Adj_N': 0.7,
    'N1->N': 0.3,
    'VP->V_NP_ADV_PP': 0.5,
    'VP->V_NP_PP': 0.2,
    'VP->V_NP_ADV': 0.15,
    'VP->V_NP': 0.15,
    'Pron=i': 0.5,
    'Pron=you': 0.35,
    'V=hope': 0.6,
    'V=enjoy': 0.5,
    'V=got': 0.6,
    'Word=review': 0.6,
    'Word=content': 0.6,
}

SMOOTH_K = 5e-4
TEMPERATURE = 1.0

def generate_candidates(tokens):
    edit_ops = [
        'insert_I',
        'delete_first_too',
        'delete_to_before_enjoy',
        'move_too_after_it',
        'sub_as_to_with',
        'sub_this_to_your',
        'sub_see_to_review',
        'sub_last_message_to_content'
    ]

    def apply_flags(toks, flags):
        toks = toks.copy()
        # substitutions
        if flags.get('sub_as_to_with'):
            toks = ['with' if x == 'as' else x for x in toks]
        if flags.get('sub_this_to_your'):
            toks = ['your' if x == 'this' else x for x in toks]
        if flags.get('sub_see_to_review'):
            toks = ['review' if x == 'see' else x for x in toks]
        if flags.get('sub_last_message_to_content'):
            # replace last 'message' with 'content'
            for i in range(len(toks)-1, -1, -1):
                if toks[i] == 'message':
                    toks[i] = 'content'
                    break

        # deletions / moves
        if flags.get('delete_first_too'):
            if 'too' in toks:
                toks.remove('too')
        if flags.get('delete_to_before_enjoy'):
            for i in range(len(toks)-1):
                if toks[i] == 'to' and toks[i+1] == 'enjoy':
                    toks.pop(i)
                    break
        if flags.get('move_too_after_it'):
            if 'too' in toks:
                toks.remove('too')
            if 'it' in toks:
                idx = toks.index('it')
                toks.insert(idx+1, 'too')
            else:
                toks.append('too')

        if flags.get('insert_I'):
            toks = ['i'] + toks

        return toks

    candidates = {}
    for mask in range(1 << len(edit_ops)):
        flags = {op: bool(mask & (1 << i)) for i, op in enumerate(edit_ops)}
        cand = tuple(apply_flags(tokens, flags))
        candidates[cand] = flags  # keep one flag set per unique tokens
    candidates[tuple(tokens)] = {op: False for op in edit_ops}  # include original
    return [(list(k), v) for k, v in candidates.items()]


# small unigram-like backoff distribution from BIGRAM totals
_UNIGRAM_SUM = {}
_TOTAL = 0.0
for (a, b), score in BIGRAM.items():
    _UNIGRAM_SUM[a] = _UNIGRAM_SUM.get(a, 0.0) + score
    _TOTAL += score
UNIGRAM_BACKOFF = {}
if _TOTAL > 0:
    for a, c in _UNIGRAM_SUM.items():
        UNIGRAM_BACKOFF[a] = c / _TOTAL

def bigram_cond_prob(prev, w, vocab_list=VOCAB):
    raw = BIGRAM.get((prev, w), 0.0)
    # approximate denominator by summing explicit bigram masses for this prev over our vocab
    denom = sum(BIGRAM.get((prev, x), 0.0) for x in vocab_list)
    V = max(len(vocab_list), 50)
    smoothed = (raw + SMOOTH_K) / (denom + SMOOTH_K * V)
    # interpolate
    alpha = 0.15
    back = UNIGRAM_BACKOFF.get(prev, 1.0 / V)
    return (1.0 - alpha) * smoothed + alpha * back

def bigram_score(tokens, temp=TEMPERATURE):
    if not tokens:
        return 1.0
    logp = 0.0
    prev = '<s>'
    for w in tokens:
        p = bigram_cond_prob(prev, w)
        logp += math.log(max(p, 1e-300))
        prev = w
    logp = logp / temp
    return math.exp(logp)

def structure_score(tokens):
    if not tokens:
        return 1e-12
    prob = 1.0
    first = tokens[0]
    if first in ('i', 'you', 'we', 'it'):
        prob *= PCFG_PROBS.get('S->NP_VP', 0.5) * PCFG_PROBS.get('NP->Pron', 0.35)
        prob *= PCFG_PROBS.get(f'Pron={first}', 0.1) if f'Pron={first}' in PCFG_PROBS else 0.5
    else:
        prob *= PCFG_PROBS.get('S->VP_NP', 0.2)
    # lexical boosts
    for t in tokens:
        if PCFG_PROBS.get(f'V={t}') is not None:
            prob *= PCFG_PROBS[f'V={t}']
        if PCFG_PROBS.get(f'Word={t}') is not None:
            prob *= PCFG_PROBS[f'Word={t}']
    # reward target chunks
    chunk1 = ['with', 'my', 'deepest', 'wishes']
    if any(tokens[i:i+4] == chunk1 for i in range(len(tokens)-3)):
        prob *= PCFG_PROBS.get('VP->V_NP_ADV_PP', 0.5)
    chunk2 = ['to', 'review', 'the', 'approved', 'content']
    if any(tokens[i:i+5] == chunk2 for i in range(len(tokens)-4)):
        prob *= 1.3
    return max(prob, 1e-12)


def softmax_from_logits(logits, temp=1.0):
    mx = max(logits)
    exps = [math.exp((x - mx) / temp) for x in logits]
    s = sum(exps)
    return [e / s for e in exps]

def rank_and_sample_candidates(scored_candidates, sample_n=0, sample_temp=1.0):
    logits = [c['score'] for c in scored_candidates]
    posterior = softmax_from_logits(logits, temp=sample_temp)
    for c, p in zip(scored_candidates, posterior):
        c['posterior'] = p
    sorted_cands = sorted(scored_candidates, key=lambda x: x['score'], reverse=True)
    sampled = []
    if sample_n > 0:
        for _ in range(sample_n):
            r = random.random()
            acc = 0.0
            for i, p in enumerate(posterior):
                acc += p
                if r <= acc:
                    sampled.append(scored_candidates[i])
                    break
    return sorted_cands, sampled

def rewrite_sentence(src_sentence, alpha=1.0, beta=1.0, gamma=1.0, temp=TEMPERATURE,
                     sample_n=0, sample_temp=1.0):
    src_tokens = tokenize(src_sentence)
    cands = generate_candidates(src_tokens)
    scored = []
    for toks, flags in cands:
        struct_p = structure_score(toks)
        bigram_p = bigram_score(toks, temp=temp)
        ed = levenshtein(src_tokens, toks)
        # log components
        struct_log = math.log(max(struct_p, 1e-300))
        bigram_log = math.log(max(bigram_p, 1e-300))
        # combined log-score (higher is better)
        combined_log = alpha * struct_log + beta * bigram_log - gamma * ed
        scored.append({
            'tokens': toks,
            'flags': flags,
            'structure_prob': struct_p,
            'structure_log': struct_log,
            'bigram_prob': bigram_p,
            'bigram_log': bigram_log,
            'edit_cost': ed,
            'score': combined_log
        })

    sorted_cands, sampled = rank_and_sample_candidates(scored, sample_n=sample_n, sample_temp=sample_temp)
    best = sorted_cands[0] if sorted_cands else None
    return best, sorted_cands, sampled

if __name__ == "__main__":
    inputs = [
        "Hope you too, to enjoy it as my deepest wishes",
        "I got this message to see the approved message"
    ]

    alpha = 1.0    # structure weight
    beta = 1.0     # bigram weight
    gamma = 0.8    # edit penalty weight
    temp = 1.0     # bigram temperature
    sample_n = 5   # number of stochastic samples to draw (0 disables)
    sample_temp = 0.8  # sampling softmax temperature

    outputs = []
    for src in inputs:
        best, ranked, sampled = rewrite_sentence(src, alpha=alpha, beta=beta, gamma=gamma, temp=temp,
                                                 sample_n=sample_n, sample_temp=sample_temp)
        print("INPUT:", src)
        if best:
            print("BEST (deterministic argmax):", detokenize(best['tokens']))
            print("  breakdown: struct_log={:.4f}, bigram_log={:.4f}, edit_cost={}, combined_score={:.4f}, posterior={:.4f}".format(
                best['structure_log'], best['bigram_log'], best['edit_cost'], best['score'], best.get('posterior', 0.0)
            ))
            outputs.append(detokenize(best['tokens']))
        else:
            print("No candidate produced.")
        print("\nTop 6 candidates (detokenized) with scores and posterior:")
        for c in ranked[:6]:
            print("  {: .4f}  ed={}  p_struct={:.4g} p_bigram={:.4g} posterior={:.4g} -> {}".format(
                c['score'], c['edit_cost'], c['structure_prob'], c['bigram_prob'], c.get('posterior', 0.0),
                detokenize(c['tokens'])
            ))
        if sampled:
            print("\nSampled outputs (stochastic):")
            for s in sampled:
                print("  posterior={:.4g} -> {}".format(s.get('posterior', 0.0), detokenize(s['tokens'])))
        print("-" * 80)

    with open("outputs_of_A.txt", "w") as f:
        for i, output in enumerate(outputs):
            f.write(f"Output {i+1}:\n{output}\n\n")

    print("All outputs have been written to outputs_of_A.txt")

INPUT: Hope you too, to enjoy it as my deepest wishes
BEST (deterministic argmax): I hope you enjoy it too, with my deepest wishes.
  breakdown: struct_log=-3.7297, bigram_log=-2.6965, edit_cost=5, combined_score=-10.4262, posterior=0.6467

Top 6 candidates (detokenized) with scores and posterior:
  -10.4262  ed=5  p_struct=0.024 p_bigram=0.06744 posterior=0.6467 -> I hope you enjoy it too, with my deepest wishes.
  -10.9356  ed=4  p_struct=0.03 p_bigram=0.01457 posterior=0.3421 -> hope you enjoy it too, with my deepest wishes.
  -14.0693  ed=4  p_struct=0.024 p_bigram=0.000793 posterior=0.006807 -> I hope you enjoy it, with my deepest wishes.
  -14.5787  ed=3  p_struct=0.03 p_bigram=0.0001713 posterior=0.003601 -> hope you enjoy it, with my deepest wishes.
  -16.3040  ed=3  p_struct=0.048 p_bigram=1.907e-05 posterior=0.0004167 -> I hope you enjoy it as my deepest wishes.
  -16.8134  ed=2  p_struct=0.06 p_bigram=4.119e-06 posterior=0.0002205 -> hope you enjoy it as my deepest wishes.



B. Ανακατασκευή του συνόλου των 2 κειμένων με χρήση 3 διαφορετικών αυτόματων βιβλιοθηκών python pipelines.

In [1]:
from transformers import pipeline

def paraphrase_text(include_paraphrase, include_dot, cut_at_question_mark, pipeline, input_text, **pipeline_args):
    print("-" * 50 + "\n")
    print("Original Text:\n" + input_text + "\n")
    if not cut_at_question_mark: sentences = input_text.split('. ')
    else: sentences = input_text.replace('? ', '. ').split('. ')
    corrected_sentences = []

    print("Processing sentences: \n")
    bar_len = 30
    filled_len = 0
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    print(f"\r[{bar}] 0/{len(sentences)}")
    for idx, sentence in enumerate(sentences):
        if include_dot and not sentence.endswith('.'): sentence += '.'
        if include_paraphrase: sentence = "paraphrase: " + sentence
        out = pipeline(sentence, **pipeline_args)
        corrected_sentences.append(out[0]['generated_text'].strip())
        progress = (idx + 1) / len(sentences)
        filled_len = int(bar_len * progress)
        bar = '=' * filled_len + '-' * (bar_len - filled_len)
        print(f"\r[{bar}] {idx + 1}/{len(sentences)}")

    print("\nCorrected Text:")
    final_text = ' '.join(corrected_sentences)
    print(final_text + "\n")
    return final_text


pipeline1 = pipeline(
    "text2text-generation",
    model="humarin/chatgpt_paraphraser_on_T5_base",
    tokenizer="humarin/chatgpt_paraphraser_on_T5_base"
)

pipeline2 = pipeline(
    "text2text-generation",
    model="stanford-oval/paraphraser-bart-large",
    tokenizer="stanford-oval/paraphraser-bart-large"
)

pipeline3 = pipeline(
    "text2text-generation",
    model="tuner007/pegasus_paraphrase",
    tokenizer="tuner007/pegasus_paraphrase"
)



input_text1 = ("Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in "
              "our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the "
              "doctor, as his next contract checking, to all of us. I got this message to see the approved message. "
              "In fact, I have received the message from the professor, to show me, this, a couple of days ago. "
              "I am very appreciated the full support of the professor, for our Springer proceedings publication")

input_text2 = ("During our final discuss, I told him about the new submission — the one we were waiting since last "
              "autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe "
              "editor? Anyway, I believe the team, although bit delay and less communication at recent days, they "
              "really tried best for paper and cooperation. We should be grateful, I mean all of us, for the "
              "acceptance and efforts until the Springer link came finally last week, I think. Also, kindly remind me "
              "please, if the doctor still plan for the acknowledgments section edit before he sending again. "
              "Because I didn’t see that part final yet, or maybe I missed, I apologize if so. Overall, "
              "let us make sure all are safe and celebrate the outcome with strong coffee and future targets")


outputs = []
print("Using Model 1: humarin/chatgpt_paraphraser_on_T5_base\n")
outputs.append(paraphrase_text(True, False, False, pipeline1, input_text1))
outputs.append(paraphrase_text(True,  False, False, pipeline1, input_text2, do_sample = True, temperature=0.8))

print("Using Model 2: stanford-oval/paraphraser-bart-large\n")
outputs.append(paraphrase_text(False, True, True, pipeline2, input_text1))
outputs.append(paraphrase_text(False, True, True, pipeline2, input_text2))
print("Using Model 3: tuner007/pegasus_paraphrase\n")
outputs.append(paraphrase_text(True, False, False, pipeline3, input_text1,do_sample = True, temperature=1.1 ,num_beams=10, repetition_penalty=2.0, top_k=30, top_p=0.8))
outputs.append(paraphrase_text(True, False, True, pipeline3, input_text2, do_sample = True, temperature=1.1 ,num_beams=10, repetition_penalty=2.0, top_k=30, top_p=0.8))

with open("outputs_of_B.txt", "w") as f:
    for i, output in enumerate(outputs):
        f.write(f"Output {i+1}:\n{output}\n\n")

print("All outputs have been written to outputs_of_B.txt")

Device set to use cpu
Device set to use cpu
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Using Model 1: humarin/chatgpt_paraphraser_on_T5_base

--------------------------------------------------

Original Text:
Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication

Processing sentences: 

[------------------------------] 0/6
[=====-------------------------] 1/6
[==========--------------------] 2/6
[===============---------------] 3/6
[====================----------] 4/6
[=========================-----] 5/6
[==============================] 6/6

Corrected Text:
Our Chinese culture celebrates the dragon boat festival today to ensure the 

C. Συγκρίνετε τα αποτελέσματα της κάθε προσέγγισης με τις κατάλληλες τεχνικές

In [2]:
import re
from nltk import PCFG
from nltk.parse import ViterbiParser

# Define grammar
grammar = PCFG.fromstring(r"""
# Top-level S
S -> NP VP [0.60] | VP [0.20] | Modal NP VP [0.10] | Modal VP [0.05] | NP [0.05]

# Noun phrases
NP -> Pron [0.18] | Det N1 [0.18] | N [0.12] | Det Adj N [0.18] | Det N [0.18] | NP PP [0.16]
N1 -> Adj N [0.40] | N [0.30] | N INF [0.20] | N SBAR [0.10]

# Verb phrases (probabilities sum to 1.0)
VP -> V NP [0.23]
VP -> V [0.05]
VP -> V NP PP [0.11]
VP -> V NP INF [0.08]
VP -> V S [0.03]
VP -> Aux V NP [0.10]
VP -> Aux V INF [0.04]
VP -> Modal V NP [0.03]
VP -> Modal V INF [0.02]
VP -> Adv VP [0.03]
VP -> V NP Adv [0.04]
VP -> V NP SBAR [0.10]
VP -> V NP SBAR SBAR [0.04]
VP -> V NP Adv PP [0.06]
VP -> Aux PP [0.04]

# Infinitival complement
INF -> 'to' VP [1.0]

# Subordinate / comparative clauses
SBAR -> Prep S [0.80] | Prep VP [0.15] | Prep Adv Prep S [0.05]

# Prepositional phrase
PP -> Prep NP [1.0]
Prep -> 'as' [0.34] | 'so' [0.33] | 'with' [0.33]

# Lexicon / preterminals
Pron -> 'i' [0.125] | 'you' [0.125] | 'we' [0.125] | 'it' [0.125] | 'my' [0.125] | 'me' [0.125] | 'this' [0.125] | 'one' [0.125]

Det -> 'the' [0.20] | 'my' [0.20] | 'your' [0.20] | 'a' [0.20] | 'this' [0.20]

Adj -> 'deepest' [0.20] | 'approved' [0.20] | 'good' [0.20] | 'sanctioned' [0.20] | 'much' [0.20]

N -> 'review' [0.10] | 'content' [0.10] | 'wishes' [0.10] | 'wish' [0.10] | 'message' [0.10]
N -> 'it' [0.10] | 'time' [0.10] | 'thoughts' [0.10] | 'report' [0.10] | 'one' [0.10]

V -> 'hope' [0.083333] | 'enjoy' [0.083333] | 'got' [0.083333] | 'review' [0.083333]
V -> 'approve' [0.083333] | 'have' [0.083333] | 'notified' [0.083333] | 'view' [0.083333]
V -> 'received' [0.083333] | 'see' [0.083333] | 'do' [0.083333] | 'is' [0.083333]

Modal -> 'may' [0.5] | 'could' [0.5]

Aux -> 'was' [0.25] | 'have' [0.25] | 'do' [0.25] | 'are' [0.25]

Adv -> 'too' [0.3333] | 'also' [0.3333] | 'much' [0.3333]
""")

parser = ViterbiParser(grammar)

# Read sentences from file
sentences = []
with open('outputs_of_A.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line not in ["Output 1:", "Output 2:", ""]:
            sentences.append(line)

with open('outputs_of_B.txt', 'r', encoding='utf-8') as f:
    flag = True
    output = 0
    for line in f:
        line = line.strip()
        if line not in ["Output 1:", "Output 2:", "Output 3:", "Output 4:", "Output 5:", "Output 6:", ""]:
            if flag:
                sentences_in_line = re.split(r'(?<=[.!?])\s+', line)
                if output == 1:
                    sentences.append(sentences_in_line[2])
                    sentences.append(sentences_in_line[4])
                else:
                    sentences.append(sentences_in_line[1])
                    sentences.append(sentences_in_line[3])
                flag = False
                output += 1
            else:
                flag = True

def clean_and_tokenize(sent):
    # Remove punctuation and lowercase
    clean = re.sub(r"[^\w\s]", "", sent).lower()
    return clean.split()

# Parse each sentence
for sentence in sentences:
    tokens = clean_and_tokenize(sentence)
    print(f"\nParsing sentence: {' '.join(tokens)}")

    try:
        grammar.check_coverage(tokens)
    except ValueError as e:
        print("Coverage error:", e)
        continue

    parses = list(parser.parse(tokens))
    if not parses:
        print("No valid parse found.")
    else:
        for tree in parses:
            tree.pretty_print()
            break



Parsing sentence: i hope you enjoy it too with my deepest wishes
      S                                                  
  ____|__________                                         
 |               VP                                      
 |     __________|_________                               
 |    |                    S                             
 |    |     _______________|___                           
 |    |    |                   VP                        
 |    |    |      _____________|________                  
 |    |    |     |    |    |            PP               
 |    |    |     |    |    |    ________|_____            
 NP   |    NP    |    NP   |   |              NP         
 |    |    |     |    |    |   |     _________|______     
Pron  V   Pron   V   Pron Adv Prep Det       Adj     N   
 |    |    |     |    |    |   |    |         |      |    
 i   hope you  enjoy  it  too with  my     deepest wishes


Parsing sentence: i got your message to review the appr